In [1]:
import sys
sys.path.append('/app')


In [2]:
import requests
import json
from datetime import datetime
from typing import Optional, List, Dict, Any, Generator


from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

BASE_URL = "http://localhost:8765"
headers = {"Authorization": f"Bearer {vexa.token}", "Content-Type": "application/json"}

def submit_token(token: str) -> Dict:
    return requests.post(f"{BASE_URL}/submit_token", json={"token": token}).json()

def chat(query: str, thread_id: Optional[str] = None, model: str = "gpt-4o-mini", temperature: float = 0.7) -> Dict:
    response = requests.post(f"{BASE_URL}/chat", headers=headers, json={
        "query": query, "thread_id": thread_id, "model": model, "temperature": temperature
    }, stream=True)
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}
def global_search(query: str, limit: int = 200, min_score: float = 0.4) -> Dict:
    return requests.post(f"{BASE_URL}/search/global", headers=headers, 
        json={"query": query, "limit": limit, "min_score": min_score}).json()

def search_transcripts(query: str, meeting_ids: Optional[List[str]] = None, min_score: float = 0.8) -> Dict:
    return requests.post(f"{BASE_URL}/search/transcripts", headers=headers,
        json={"query": query, "meeting_ids": meeting_ids, "min_score": min_score}).json()

def get_meetings(offset: int = 0, limit: int = 50) -> Dict:
    return requests.get(f"{BASE_URL}/meetings/all", headers=headers, 
        params={"offset": offset, "limit": limit}).json()

def get_meeting_details(meeting_id: str) -> Dict:
    return requests.get(f"{BASE_URL}/meeting/{meeting_id}/details", headers=headers).json()

def create_share_link(target_email: str, access_level: str = "READ", 
                     expiration_hours: int = 24, include_existing: bool = True) -> Dict:
    return requests.post(f"{BASE_URL}/share-links", headers=headers, json={
        "access_level": access_level, "target_email": target_email,
        "expiration_hours": expiration_hours, "include_existing_meetings": include_existing
    }).json()

def accept_share_link(token: str, accepting_email: str) -> Dict:
    return requests.post(f"{BASE_URL}/share-links/accept", headers=headers,
        json={"token": token, "accepting_email": accepting_email}).json()

def start_indexing(num_meetings: int = 200) -> Dict:
    return requests.post(f"{BASE_URL}/start_indexing", headers=headers,
        json={"num_meetings": num_meetings}).json()

def get_indexing_status() -> Dict:
    return requests.get(f"{BASE_URL}/indexing_status", headers=headers).json()

# Add these functions to the initialization cell with other API functions
def get_threads(meeting_id: Optional[str] = None) -> Dict:
    """Get threads for global or meeting-specific context"""
    if meeting_id:
        return requests.get(f"{BASE_URL}/threads/{meeting_id}", headers=headers).json()
    return requests.get(f"{BASE_URL}/threads", headers=headers).json()

def delete_thread(thread_id: str) -> Dict:
    return requests.delete(f"{BASE_URL}/thread/{thread_id}", headers=headers).json()

def chat_meeting(
    query: str, 
    meeting_ids: List[str], 
    thread_id: Optional[str] = None, 
    model: str = "gpt-4o-mini", 
    temperature: float = 0.7
) -> Dict:
    response = requests.post(
        f"{BASE_URL}/chat/meeting", 
        headers=headers, 
        json={
            "query": query,
            "meeting_ids": meeting_ids,
            "thread_id": thread_id,
            "model": model,
            "temperature": temperature
        },
        stream=True
    )
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}

def submit_token(token: str) -> Dict:
    response = requests.post(f"{BASE_URL}/submit_token", json={"token": token})
    result = response.json()
    print(f"User ID: {result.get('user_id')}")
    print(f"User Name: {result.get('user_name')}")
    print(f"Image URL: {result.get('image')}")
    return result

# Example usage:
# r = submit_token(vexa.token)
# Add this function to your test notebook
def get_meetings_by_speakers(speakers: List[str], limit: int = 50, offset: int = 0) -> Dict:
    return requests.post(
        f"{BASE_URL}/meetings/by-speakers",
        headers=headers,
        json={"speakers": speakers, "limit": limit, "offset": offset}
    ).json()

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [3]:
def chat_meeting_summary(
    query: str,
    meeting_ids: List[str],
    include_discussion_points: bool = True,
    thread_id: Optional[str] = None,
    model: str = "gpt-4o-mini",
    temperature: float = 0.7
) -> Dict:
    response = requests.post(
        f"{BASE_URL}/chat/meeting/summary",
        headers=headers,
        json={
            "query": query,
            "meeting_ids": meeting_ids,
            "include_discussion_points": include_discussion_points,
            "thread_id": thread_id,
            "model": model,
            "temperature": temperature
        },
        stream=True
    )
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}

# Example usage:
"""
# Get meetings for a specific speaker
meetings = get_meetings_by_speakers(['Olga Nemirovskaya'])
meeting_ids = [m['meeting_id'] for m in meetings['meetings']]

# Chat about the meetings using summaries
response = chat_meeting_summary(
    query="What were the main topics discussed in these meetings?",
    meeting_ids=meeting_ids,
    include_discussion_points=True
)
"""

'\n# Get meetings for a specific speaker\nmeetings = get_meetings_by_speakers([\'Olga Nemirovskaya\'])\nmeeting_ids = [m[\'meeting_id\'] for m in meetings[\'meetings\']]\n\n# Chat about the meetings using summaries\nresponse = chat_meeting_summary(\n    query="What were the main topics discussed in these meetings?",\n    meeting_ids=meeting_ids,\n    include_discussion_points=True\n)\n'

In [6]:
global_search('url')

{'results': [{'topic_name': 'Настройка Redirect URL для Google',
   'speaker_name': 'Artem Puzik',
   'summary': 'Артем Пузик работает над настройкой Redirect URL для Google.',
   'details': 'Артем Пузик упоминает, что он добавил Redirect URL для Google в проект, что является частью его работы над фронтендом.',
   'meeting_id': '66528ad8-a00c-41e2-adfa-6ee53c18b7ff',
   'timestamp': '2024-11-20T14:40:59.576000',
   'score': 1.0764239,
   'source': 'main'},
  {'topic_name': 'Base.url',
   'speaker_name': 'Artem Puzik',
   'summary': 'Base URL configuration for the project.',
   'details': 'Artem states that he added a Base.url for their project, which is essential for its functionality.',
   'meeting_id': '66528ad8-a00c-41e2-adfa-6ee53c18b7ff',
   'timestamp': '2024-11-20T14:40:59.576000',
   'score': 0.93431525,
   'source': 'main'},
  {'topic_name': 'chat.dev.vets.ai',
   'speaker_name': 'Dmitriy Grankin',
   'summary': 'Another URL related to the project.',
   'details': 'Dmitriy ref

In [4]:
meetings = get_meetings_by_speakers(['Olga Nemirovskaya'])
meeting_ids = [m['meeting_id'] for m in meetings['meetings']]


In [13]:
meeting_ids

['7973854d-ce5a-4263-b0f5-74007ea7a630',
 '06e239a1-dbc9-4e3d-968c-a307f30c0adc',
 '1be62b63-99ea-4aea-b284-a2ed45e44b82',
 'fb2aea72-f75d-4790-99dc-bbc4c7219a31',
 'c6bd2072-dcd6-45b9-8006-7b975bfe43ad',
 '08a4b53e-6ff1-4a95-9263-57ca27f0995a',
 'ce18c6b7-5636-4e97-a5e5-726f4ef9649c',
 '9178c8e7-da6a-4883-9fe1-cc4ee30125ea',
 '04de7398-49fd-4a57-962f-2798fc080b04',
 'cac618cd-fb7f-4d15-ada4-0f06358b13e5',
 'b3d139a0-cc7a-4fa0-9f29-f69d5dc82c34',
 'e941833d-5234-4f52-9a2a-d68f9ba45e13',
 'f3dd4aba-acde-46e0-9b6c-12591c740e0e',
 'e3dea53c-a0d6-4b10-ab63-8b4c120bd245',
 '3a964fa0-9463-4e7b-ad12-279ad64717d0',
 '46a1dd08-0198-4aaf-84d5-27b1b1b87a94',
 '3ead8b34-f5de-42ef-8d28-5dd64a2b8669',
 '99e22c95-9f1c-4704-b6ab-394e5d54e970',
 'b1c57a93-7750-4633-a579-4d374d9c99f5',
 'd06b1c93-f225-40cd-8448-e9efd790336e',
 'd7d8d9e4-f349-42b3-9599-f9707f1405f8',
 '331c3dbb-9233-4894-ad9d-6a537df88b7b',
 '0eceb16a-7732-4e91-931b-ab588771dd41',
 '55fb40e1-096c-4280-b205-aa4cc68f6c76',
 '40001390-7187-

In [14]:
# response = chat_meeting_summary(
#     query="more on number 5",
#     thread_id='e85533f2-da28-4ee6-8849-d6b71626bcdc',
#     meeting_ids=meeting_ids,
#     include_discussion_points=True
# )

In [3]:
# Get a specific meeting with discussion points
meeting = get_meeting_details('0c743875-dd3b-498c-ada8-6b75d45542ed')  # Meeting with Olga Nemirovskaya
print("\nMeeting Details:")
print(f"Name: {meeting['meeting_name']}")
print(f"Timestamp: {meeting['timestamp']}")
print("\nDiscussion Points:")
for dp in meeting.get('discussion_points', []):
    print(f"\nTopic: {dp['topic_name']}")
    print(f"Summary: {dp['summary']}")
    print(f"Details: {dp['details']}")
    print(f"Referenced Text: {dp['referenced_text']}")
    print(f"Speaker: {dp['speaker_id']}")
    print(f"Type: {dp['topic_type']}")

NameError: name 'get_meeting_details' is not defined

In [4]:
print(meeting)


NameError: name 'meeting' is not defined

In [17]:
df = pd.DataFrame(get_meetings()['meetings'])

NameError: name 'pd' is not defined

In [11]:
df['timestamp'] = pd.to_datetime(df['timestamp'],format='mixed')

In [19]:
print(get_meeting_details('26045454-6226-4ecb-bfc4-46a3a1e219eb'))


{'meeting_id': '26045454-6226-4ecb-bfc4-46a3a1e219eb', 'timestamp': '2024-07-23T09:17:26.198151', 'meeting_name': '09:17', 'transcript': '[{"speaker": "Inferno Maverick", "speaker_id": "TBD", "content": " attenzione veloce", "html_content": null, "html_content_short": null, "keywords": [], "timestamp": "2024-07-23T09:17:26.198151Z"}, {"speaker": "click to name speaker", "speaker_id": "TBD", "content": " Good morning, everybody\'s here, I\'m Andre, you\'ll be missing, how are you today?", "html_content": null, "html_content_short": null, "keywords": [], "timestamp": "2024-07-23T09:17:59.824151Z"}, {"speaker": "click to name speaker", "speaker_id": "TBD", "content": " Good, but it\'s a little bit hard to hear you.", "html_content": null, "html_content_short": null, "keywords": [], "timestamp": "2024-07-23T09:18:05.584151Z"}, {"speaker": "click to name speaker", "speaker_id": "TBD", "content": " Yeah, any better?", "html_content": null, "html_content_short": null, "keywords": [], "timesta

In [22]:
get_meeting_details('0c743875-dd3b-498c-ada8-6b75d45542ed')

{'meeting_id': '0c743875-dd3b-498c-ada8-6b75d45542ed',
 'timestamp': '2024-11-20T16:40:18.920000',
 'meeting_name': 'Языковые барьеры и проблемы с тредами в чате',
 'transcript': '[{"speaker": "", "speaker_id": "TBD", "content": " \\u0420\\u0435\\u043a\\u043e\\u0440\\u0434.", "html_content": "\\u0420\\u0435\\u043a\\u043e\\u0440\\u0434.", "html_content_short": "", "keywords": [], "timestamp": "2024-11-20T16:40:18.920000Z"}, {"speaker": "Dmitriy Grankin", "speaker_id": "TBD", "content": " \\u0422\\u0430\\u043c, \\u043f\\u0440\\u0430\\u0432\\u0434\\u0430, \\u0438\\u043d\\u0434\\u0435\\u043a\\u0441\\u0430\\u0446\\u0438\\u044f \\u043f\\u043e\\u043a\\u0430 \\u043d\\u0435 \\u0441\\u043e\\u0432\\u0441\\u0435\\u043c \\u043d\\u0430\\u0441\\u0442\\u0440\\u043e\\u0435\\u043d\\u0430.", "html_content": "\\u0422\\u0430\\u043c, \\u043f\\u0440\\u0430\\u0432\\u0434\\u0430, <b>\\u0438\\u043d\\u0434\\u0435\\u043a\\u0441\\u0430\\u0446\\u0438\\u044f</b> \\u043f\\u043e\\u043a\\u0430 \\u043d\\u0435 \\u0441\\u